# MEMS-ONLY

## Introduction

In this notebook we program the MEMS-ONLY PCB, which has 2 STHV1600 ICs for pulsing

In [1]:
#import of all libraries
import threading
import logging
import asyncio
import time
import numpy as np
from pynq import Overlay
from pynq.overlays.ip import *
from pynq.overlays.ip.STHV1600 import sthv1600_regmap as reg
import pynq.interrupt

we import the bitstream as an overlay

In [2]:
#ol = Overlay('/home/xilinx/pynq/overlays/MEMS_ONLY/base.bit')
ol = Overlay('/usr/local/lib/python3.6/dist-packages/carpatchiot_mems_only/bitstream/mems_only.bit')
# ol?

In [3]:
#sthv is the main point of contact
leds = ol.gpio_leds
switches = ol.gpio_switches
buttons = ol.gpio_buttons
axi_intr_c = ol.axi_intc_0
BRAM = ol.axi_bram_ctrl_0
sthv = ol.STHV1600_0
prg = ol.STHV1600_0.S00_AXI_PRG
trg = ol.STHV1600_0.S01_AXI_TRIG
intr = ol.STHV1600_0.S_AXI_INTR
resetn = ol.axi_gpio_0
# sthv?
sthv.connect_BRAM(BRAM)
print(pynq.ps.Clocks.fclk0_mhz)
rc = pynq.overlays.ip.STHV1600.mems_only_channels.ROW_COLUMN()
read_channels = rc.channels_read()
drive_channels = rc.channels_drive()

150.0


# Zedboard peripherals

a small use-case of the pushbuttons, switches & LEDs as feedback mech

In [16]:
outputMask = 0x0
tristateRegisterOffset = 0x4
leds.write(tristateRegisterOffset, outputMask)

def button_thread(name):
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")
    logging.info("started thread")
    while(not switches.read()):
        leds.write(0x0, buttons.read())
x = threading.Thread(target=button_thread, args=(1,))
x.start()

09:24:47: started thread


In [15]:
outputMask = 0x0
tristateRegisterOffset = 0x4
resetn.write(tristateRegisterOffset, outputMask)
resetn.write(0x0,0x0)
time.sleep(1)
resetn.write(0x0,0x1)

## interrupts

In [8]:
print(ol.interrupt_pins)

{'system_ila_0/probe5': {'controller': 'axi_intc_0', 'index': 0, 'fullpath': 'system_ila_0/probe5'}, 'STHV1600_0/irq': {'controller': 'axi_intc_0', 'index': 0, 'fullpath': 'STHV1600_0/irq'}, 'xlconcat_0/In0': {'controller': 'axi_intc_0', 'index': 0, 'fullpath': 'xlconcat_0/In0'}, 'xlconcat_0/In1': {'controller': 'axi_intc_0', 'index': 1, 'fullpath': 'xlconcat_0/In1'}, 'axi_gpio_1/ip2intc_irpt': {'controller': 'axi_intc_0', 'index': 1, 'fullpath': 'axi_gpio_1/ip2intc_irpt'}, 'axi_gpio_2/ip2intc_irpt': {'controller': 'axi_intc_0', 'index': 2, 'fullpath': 'axi_gpio_2/ip2intc_irpt'}, 'xlconcat_0/In2': {'controller': 'axi_intc_0', 'index': 2, 'fullpath': 'xlconcat_0/In2'}, 'gpio_buttons/ip2intc_irpt': {'controller': 'axi_intc_0', 'index': 3, 'fullpath': 'gpio_buttons/ip2intc_irpt'}, 'xlconcat_0/In3': {'controller': 'axi_intc_0', 'index': 3, 'fullpath': 'xlconcat_0/In3'}, 'xlconcat_0/In4': {'controller': 'axi_intc_0', 'index': 4, 'fullpath': 'xlconcat_0/In4'}, 'gpio_switches/ip2intc_irpt': {

In [9]:
axi_intr = pynq.interrupt.Interrupt(pinname='STHV1600_0/irq')
# event = asyncio.Event()
async def interrupt_test():
    print("in begin interrupt test")
    await axi_intr.wait()
    axi_intr.event.clear()
    print("STHV1600 interrupt triggered!")

async def interrupt_thread():
    print("start thread")
#     while(not axi_intr_c.read()):
#         wait_var=0
    await axi_intr.wait()
    axi_intr.event.clear()
    result_available.set()
    wait_var=1
    
def axi_interrupt(name, loop):
    print("start interrupt loop", name)
    print(pynq.interrupt.get_uio_irq(irq=61))        
    print(pynq.interrupt.get_uio_device(dev_name="fabric"))

# loop = asyncio.get_event_loop()
# task = asyncio.run_coroutine_threadsafe(interrupt_test(), loop)
# print(task.result)
# global wait_var
# wait_var=0
# result_available= threading.Event()
# x = threading.Thread(target=interrupt_thread)
# x.start()  
# # loop.

def STHV_RAMREAD_INT():
    print("beginning reading")
    prg.connect_BRAM(BRAM)
    prg.read_RAM(2,  reg.DEVID0, reg.config_1)
    while(not result_available.wait()):
        time.sleep(0.1)
    print("done")
#     axi_interrupt()
# STHV_RAMREAD_INT()
# axi_interrupt()

## Programming

ways of programming:

1. least preferred way: (access via prg is more error prone, as you can easilier program the wrong values at the wrong index, as in, use the program trigger function for the PRG AXI port)

`prg.program_RAM(0X1,0x0,0X1009,[0X3])`

2. preffered way:

`sthv.STHV_PRG_RAM(amount=0x1,devId=0,address= 0x1009,data= [0x3])`

`sthv.STHV_TRIG(tx_rxtime=0x140, idle_time=0x200, cycle_amount=4)`

In [10]:
sthv.initRAM(devId=reg.DEVID0)
sthv.initRAM(devId=reg.DEVID1)

In [17]:
devIdList = [reg.DEVID0, reg.DEVID1]
sthv.example_case(devIdList=devIdList, topology='MEMS-ONLY', cycle_amount=1000000)

time_reg_ticks [25, 20, 0, 0]
start delay: [1, 22, 3, 14, 25]
amount of pulse regs: 3
--> pulse shape registers per channel:  0x4996
--> pulse shape registers per channel:  0x14f
--> pulse shape registers per channel:  0xe
-->reading from reg 0x80440
read 0x 0x2
start delay: [1, 22, 3, 14, 25]
amount of pulse regs: 3
--> pulse shape registers per channel:  0x4996
--> pulse shape registers per channel:  0x14f
--> pulse shape registers per channel:  0xe
-->reading from reg 0x80444
read 0x 0x2
MEMS-ONLY programming
programming STHV1600 trigger in ticks mode 160


In [18]:
result = sthv.STHV_RAM_READ(amount=1,address=reg.channel_0,devId=reg.DEVID0)
print(hex(result[0]))

-->reading from reg 0x40
0x4996


In [19]:
def read_test():
    res = sthv.STHV_RAM_READ(0x1, reg.DEVID0, 0x1003)
    if(res[0] != 0xf7):
        print("error in data reading of STHV1600", res, type(res[0]))
    else:
        print("read is OK",hex(res[0]), "@ reg 0x1003")
        res = sthv.STHV_RAM_READ(amount=1, devId=reg.DEVID0,address=reg.config_1)
        print(hex(res[0]))
read_test()

-->reading from reg 0x801c0
read is OK 0xf7 @ reg 0x1003
-->reading from reg 0x804c0
0x1e00


In [13]:
sthv.all_channels_same_pattern(devId=reg.DEVID0, enable=reg.Channel_sharing.SHARING.value)

-->reading from reg 0x80440
read 0x 0x2


In [14]:
hex(sthv.read_interrupt(reg.DEVID0)[0])

-->reading from reg 0x81440


'0x4000'

In [15]:
clock_offset_error = 1.5
pynq.ps.Clocks.fclk0_mhz=100.0*clock_offset_error

In [16]:
clk = pynq.ps.Clocks.fclk0_mhz/clock_offset_error

In [17]:
rc = ROW_COLUMN()

NameError: name 'ROW_COLUMN' is not defined

In [16]:
rc_channels = rc.channels_read()
print(rc_channels)

{0: {0, 2048, 4114, 4126}, 1: {0, 2304, 4115, 4126}, 2: {0, 2560, 4116, 4127}, 3: {0, 2816, 4117, 4127}, 4: {0, 3072, 4128, 4118}, 5: {0, 3328, 4128, 4119}, 6: {0, 3584, 4120, 4129}, 7: {0, 3840, 4121, 4129}, 8: {2048, 1, 4114, 4126}, 9: {2304, 1, 4115, 4126}, 10: {2560, 1, 4116, 4127}, 11: {2816, 1, 4117, 4127}, 12: {3072, 1, 4128, 4118}, 13: {3328, 1, 4128, 4119}, 14: {3584, 1, 4120, 4129}, 15: {3840, 1, 4121, 4129}}


In [13]:
a = rc.channels_drive()

In [14]:
print(a)

{0: {0, 4106, 4122}, 1: {0, 256, 4122, 4107}, 2: {0, 512, 4123, 4108}, 3: {0, 768, 4123, 4109}, 4: {0, 1024, 4124, 4110}, 5: {0, 1280, 4124, 4111}, 6: {0, 1536, 4112, 4125}, 7: {0, 1792, 4125, 4113}, 8: {0, 1, 4106, 4122}, 9: {256, 1, 4122, 4107}, 10: {512, 1, 4123, 4108}, 11: {768, 1, 4123, 4109}, 12: {1024, 1, 4124, 4110}, 13: {1280, 1, 4124, 4111}, 14: {1536, 1, 4112, 4125}, 15: {1792, 1, 4125, 4113}}


In [6]:
import distutils
help(distutils)

Help on package distutils:

NAME
    distutils - distutils

MODULE REFERENCE
    https://docs.python.org/3.6/library/distutils
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    The main package for the Python Module Distribution Utilities.  Normally
    used from a setup script as
    
       from distutils.core import setup
    
       setup (...)

PACKAGE CONTENTS
    _msvccompiler
    archive_util
    bcppcompiler
    ccompiler
    cmd
    command (package)
    config
    core
    cygwinccompiler
    debug
    dep_util
    dir_util
    dist
    errors
    extension
    fancy_getopt
    file_util
    filelist
    log
    msvc9compiler
    msvccompiler
    spawn
    sysconfig
    text_file
    unix